In [1]:
pwd

'D:\\GAISSA\\energy-repo\\last_repo\\energy-ml-serving\\experiments'

In [2]:
cd ..

D:\GAISSA\energy-repo\last_repo\energy-ml-serving


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [5]:
models = ['codet5-base',
 'codet5p-220m',
 'codeparrot-small',
 'pythia-410m']

In [10]:
file_name = "results/energy_onnx-codeparrot-small.csv"
df = pd.read_csv(file_name,  )#index_col='System Time' parse_dates=['System Time'],

In [11]:
df.columns

Index(['Delta', 'Time', 'CPU_FREQUENCY_0', 'CPU_FREQUENCY_1',
       'CPU_FREQUENCY_2', 'CPU_FREQUENCY_3', 'CPU_FREQUENCY_4',
       'CPU_FREQUENCY_5', 'CPU_FREQUENCY_6', 'CPU_FREQUENCY_7', 'CPU_USAGE_0',
       'CPU_USAGE_1', 'CPU_USAGE_2', 'CPU_USAGE_3', 'CPU_USAGE_4',
       'CPU_USAGE_5', 'CPU_USAGE_6', 'CPU_USAGE_7', 'DRAM_ENERGY (J)',
       'PACKAGE_ENERGY (J)', 'PP0_ENERGY (J)', 'PP1_ENERGY (J)',
       'TOTAL_MEMORY', 'TOTAL_SWAP', 'USED_MEMORY', 'USED_SWAP'],
      dtype='object')

In [12]:
df.head()

,Delta,Time,CPU_FREQUENCY_0,CPU_FREQUENCY_1,CPU_FREQUENCY_2,CPU_FREQUENCY_3,CPU_FREQUENCY_4,CPU_FREQUENCY_5,CPU_FREQUENCY_6,CPU_FREQUENCY_7,...,CPU_USAGE_6,CPU_USAGE_7,DRAM_ENERGY (J),PACKAGE_ENERGY (J),PP0_ENERGY (J),PP1_ENERGY (J),TOTAL_MEMORY,TOTAL_SWAP,USED_MEMORY,USED_SWAP
0,0,1714982848943,2112,2112,2112,2112,2112,2112,2112,2112,...,54.359402,1.075172,4428.836060,36189.687073,28878.376709,387.73761,17005068288,12348030976,13086830592,2029027328
1,199,1714982848943,2112,2112,2112,2112,2112,2112,2112,2112,...,100.000000,4.543594,4428.843262,36189.743286,28878.416626,387.73761,17005068288,12348030976,13086683136,2030555136
2,191,1714982849152,2112,2112,2112,2112,2112,2112,2112,2112,...,24.586769,1.412857,4428.995972,36192.827087,28881.175354,387.73761,17005068288,12348030976,13095006208,2044895232
3,200,1714982849344,2112,2112,2112,2112,2112,2112,2112,2112,...,2.024513,1.429489,4429.126953,36193.230408,28881.278503,387.73761,17005068288,12348030976,13095006208,2044895232
4,200,1714982849544,2112,2112,2112,2112,2112,2112,2112,2112,...,2.265457,2.967453,4429.266418,36193.605164,28881.451355,387.73761,17005068288,12348030976,13095088128,2044895232


In [13]:
df.columns

Index(['Delta', 'Time', 'CPU_FREQUENCY_0', 'CPU_FREQUENCY_1',
       'CPU_FREQUENCY_2', 'CPU_FREQUENCY_3', 'CPU_FREQUENCY_4',
       'CPU_FREQUENCY_5', 'CPU_FREQUENCY_6', 'CPU_FREQUENCY_7', 'CPU_USAGE_0',
       'CPU_USAGE_1', 'CPU_USAGE_2', 'CPU_USAGE_3', 'CPU_USAGE_4',
       'CPU_USAGE_5', 'CPU_USAGE_6', 'CPU_USAGE_7', 'DRAM_ENERGY (J)',
       'PACKAGE_ENERGY (J)', 'PP0_ENERGY (J)', 'PP1_ENERGY (J)',
       'TOTAL_MEMORY', 'TOTAL_SWAP', 'USED_MEMORY', 'USED_SWAP'],
      dtype='object')

In [16]:
name_mapping = {"DRAM_ENERGY (J)" : "dram_energy",
                "PACKAGE_ENERGY (J)":'package_energy',
               'PP0_ENERGY (J)':"pp0_energy",
               'PP1_ENERGY (J)':"pp1_energy"}
df = df.rename(columns=name_mapping)
df.columns = df.columns.str.lower() # to lowcase

In [17]:
df.columns

Index(['delta', 'time', 'cpu_frequency_0', 'cpu_frequency_1',
       'cpu_frequency_2', 'cpu_frequency_3', 'cpu_frequency_4',
       'cpu_frequency_5', 'cpu_frequency_6', 'cpu_frequency_7', 'cpu_usage_0',
       'cpu_usage_1', 'cpu_usage_2', 'cpu_usage_3', 'cpu_usage_4',
       'cpu_usage_5', 'cpu_usage_6', 'cpu_usage_7', 'dram_energy (j)',
       'package_energy', 'pp0_energy', 'pp1_energy', 'total_memory',
       'total_swap', 'used_memory', 'used_swap'],
      dtype='object')

In [ ]:
energy_columns = ['DRAM_ENERGY (J)', 'PACKAGE_ENERGY (J)', 'CPU_USAGE_0', 'CPU_USAGE_1']

In [ ]:
def avg_metric(df: pd.DataFrame, metric_name: str):
    all_data = None
    nb_point = 0
    for metric in df.columns[1:]:
        if metric_name in metric:
            nb_point += 1
            if all_data is None:
                all_data = df[metric].copy()
            else:
                all_data += df[metric]
    return all_data/nb_point
def generate_metric_charts(csv_file):
    all_data = []
    if not os.path.exists(csv_file):
        raise ValueError(f'{csv_file} does not exist')
    df = pd.read_csv(csv_file)
    key = "PACKAGE_ENERGY (W)"
    if "CPU_ENERGY (J)" in df.columns:
        key = "CPU_ENERGY (J)"
    if "PACKAGE_ENERGY (J)" in df.columns:
        key = "PACKAGE_ENERGY (J)"
    if "SYSTEM_POWER (Watts)" in df.columns:
        key = "SYSTEM_POWER (Watts)"
    data = df[key].copy().to_list()
    if key != "CPU_POWER (Watts)" and key != "SYSTEM_POWER (Watts)":
        df[key + "_original"] = df[key].copy()
        for i in range(0, len(data)):
            if i in df[key + "_original"] and i - 1 in df[key + "_original"]:
                # diff with previous value and convert to watts
                data[i] = (data[i] - df[key + "_original"][i - 1]) * (1000 / df["Delta"][i])
            else:
                data[i] = 0
    # data = data[1:-1]
    for i in range(0, len(data)):
        all_data.append({"Time": i, "CPU_POWER (Watts)": data[i]})
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(data, label="CPU Power")
    ax.set_ylabel('watts')

    ax2 = ax.twinx()
    
    ax2.plot(avg_metric(df, "DRAM_ENERGY"), label="DRAM_ENERGY (J)", color="red")
    ax2.plot(avg_metric(df, "CPU_USAGE_0"), label="CPU USAGE (%)", color="orange")
    ax2.plot(df["USED_MEMORY"]*100/df["TOTAL_MEMORY"], label="Used Memory (%)", color="green")
    ax2.set_ylim([0, 100])
    
    ax.set(xlabel=None)
    fig.legend(loc='upper right')
    fig.tight_layout()
    plt.show()

In [ ]:
generate_metric_charts(os.path.join(os.getcwd(), 'results/energy_onnx-codeparrot-small.csv'))

In [ ]:
# Plotting time series for energy columns
fig, axs = plt.subplots(len(energy_columns), 1, figsize=(10, 8), sharex=True)

for i, col in enumerate(energy_columns):
    df[col].plot(ax=axs[i], title=col)

# Improve spacing
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
columns_to_increment = ["DRAM_ENERGY (J)","PACKAGE_ENERGY (J)", "USED_MEMORY" , "USED_SWAP"]

In [ ]:
for column in columns_to_increment:
    df[]

In [ ]:
import pandas as pd
# energibridge - DRAM_ENERGY (J),PACKAGE_ENERGY (J) USED_MEMORY USED_SWAP


df['processor_increment'] = df['Cumulative Processor Energy_0(Joules)'].diff()

# Repeat for other cumulative columns as needed
df['IA_increment'] = df['Cumulative IA Energy_0(Joules)'].diff()
df['DRAM_increment'] = df['Cumulative DRAM Energy_0(Joules)'].diff()
df['GT_increment'] = df['Cumulative GT Energy_0(Joules)'].diff()